In [2]:
%pylab inline
import numpy as np
import scipy as sp
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle
import tensorflow.contrib.distributions.python.ops as ops
from tensorflow.contrib.distributions.python.ops import operator_pd_full

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/train.txt.npy'
data_tr = np.load(dataset_tr)
dataset_te = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/test.txt.npy'
data_te = np.load(dataset_te)
vocab = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/vocab.pkl'
vocab = pickle.load(open(vocab,'r'))

In [4]:
vocab_size=len(vocab)
def onehot(data, min_length):
    return np.bincount(data, minlength=min_length)
data_tr = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_tr if np.sum(doc)!=0])
data_te = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_te if np.sum(doc)!=0])
print data_tr.shape
print data_te.shape


(11258, 1995)
(7487, 1995)


In [5]:
n_samples_tr = data_tr.shape[0]
n_samples_te = data_te.shape[0]
docs_tr = data_tr
docs_te = data_te

In [13]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        self.is_training = tf.placeholder(tf.bool, [], name='is_training')
        
        self.h_dim = float(network_architecture["n_z"])
        self.nz=network_architecture["n_z"]
        
        self.a = np.ones((self.batch_size , self.h_dim)).astype(np.float32)/self.h_dim
        
#         self.mu2 = tf.constant(0.0)
#         self.var2 = tf.constant(  np.ones(self.batch_size,self.h_dim).astype(np.float32)  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''get mean and sigma'''
            
        self.z_mean,self.log_chol = \
            self.mean_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])
        
#         self.log_chol = \
#             self.sigma_network(self.network_weights["sub_02"], 
#                                       self.network_weights["biases_sub_02"])

        # Draw one sample z from Gaussian distribution

        eps = tf.random_normal((1, self.nz), 0, 1, dtype=tf.float32)
        '''Adding RT for subtopics'''
        
        self.chol = tf.exp(self.log_chol)
        
        self.sigma = tf.batch_matmul(self.chol,self.chol,adj_y=True)
        
#         self.sigma_log_det = 2*tf.reduce_sum(tf.log(tf.diag(tf.cholesky(self.sigma))),1)
        
        temp_calc=tf.squeeze(tf.map_fn(lambda i:tf.matmul(eps,tf.gather(tf.sqrt(self.sigma) ,i) ),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        self.z = tf.add(self.z_mean, temp_calc)

        
        '''Adding reconstruction'''
        
        self.x_reconstr_mean = \
            self._generator_network(self.z,self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
        
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()

        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z*n_z])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z*n_z], dtype=tf.float32))
            }
        with tf.variable_scope("sub02"):
            all_weights['sub_02'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z*n_z])
            }
            all_weights['biases_sub_02'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z*n_z], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def mean_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']),is_training=self.is_training,updates_collections=None)    
        
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma']),is_training=self.is_training,updates_collections=None)
        
        return z_mean,tf.reshape(z_log_sigma_sq, [self.batch_size,self.nz, self.nz])
    
    def sigma_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
#         z_sigma_sq = \
#             tf.exp(tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
#                    biases['out_log_sigma'])))  
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma']),is_training=self.is_training,updates_collections=None) 
        
        return tf.reshape(z_log_sigma_sq, [self.batch_size,self.nz, self.nz])

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
        self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2']))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0),is_training=self.is_training,updates_collections=None))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 
            
#         temp_s=self.sigma.add_to_tensor(0.0)

        temp_trace=tf.squeeze(tf.map_fn(lambda i:tf.trace(tf.gather(self.sigma,i)),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        temp_log_det=tf.squeeze(tf.map_fn(lambda i:2*tf.reduce_sum(tf.log(tf.diag_part(tf.gather(self.chol,i)))),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        print temp_trace
        print temp_log_det
        
#         latent_loss = 0.5*( temp_trace+\
#         tf.reduce_sum( tf.mul(tf.div((self.mu2 - self.z_mean),self.var2),
#                   (self.mu2 - self.z_mean)),1) - self.h_dim +\
#                            tf.reduce_sum(tf.log(self.var2),1)  - tf.log(temp_det+1e-10))

        
        latent_loss = 0.5*( temp_trace +\
        tf.reduce_sum(tf.square(self.z_mean),1) - self.h_dim +\
                           tf.log(1.0) - temp_log_det )


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .8,self.is_training:True})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost = self.sess.run((self.cost),
                                        feed_dict={self.x: X
                                                   ,self.keep_prob: 1.0,self.is_training:False})
        return cost


In [14]:
tf.reset_default_graph()
cost_plot=[]
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
#             print cost
#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
                sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)

    return vae,emb

In [17]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=100, # 1st layer encoder neurons
         n_hidden_recog_2=100, # 2nd layer encoder neurons
         n_hidden_gener_1=docs_tr.shape[1], # 1st layer decoder neurons
         n_input=docs_tr.shape[1], # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.002
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
        
minibatches = create_minibatch(docs_tr.astype('float32'))
vae,emb = train(network_architecture, training_epochs=1000,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)


'''
Change log: decoder has no dropout or additional tranformations
'''

Learning Rate: 0.002
Tensor("Squeeze_1:0", shape=(200,), dtype=float32)
Tensor("Squeeze_2:0", shape=(200,), dtype=float32)
Epoch: 0001 cost= 21558446746.518524170
Epoch: 0006 cost= 18799973.450121231
Epoch: 0011 cost= 3207284.269450058
Epoch: 0016 cost= 2069350.431958454
Epoch: 0021 cost= 993731.808066694
Epoch: 0026 cost= 1927285.826745426
Epoch: 0031 cost= 1368996.354059682
Epoch: 0036 cost= 520350.424038739
Epoch: 0041 cost= 272423.991394314
Epoch: 0046 cost= 180626.413239821
Epoch: 0051 cost= 832572.747938271
Epoch: 0056 cost= 71604.890038264
Epoch: 0061 cost= 188824.411746439
Epoch: 0066 cost= 153476.202070542
Epoch: 0071 cost= 86457.187102713
Epoch: 0076 cost= 76620.693427035
Epoch: 0081 cost= 57580.622078466
Epoch: 0086 cost= 50271.285219747
Epoch: 0091 cost= 95750.663824822
Epoch: 0096 cost= 125940.983512419
Epoch: 0101 cost= 17943.740570941
Epoch: 0106 cost= 33126.058717620
Epoch: 0111 cost= 47842.151145644
Epoch: 0116 cost= 17959.998870595
Epoch: 0121 cost= 13280.369324175
Ep

'\nChange log: decoder has no dropout or additional tranformations\n'

In [18]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])

site anonymous mechanism related usenet
budget meeting president meet economy
nsa escrow clipper wiretap crypto
buy sell dealer cheaper cheap
escape serve turks village armenia
write article michael anyone andrew
clinton libertarian liberty waco arabs
necessary maintain right clear individual
blood people suffer man die
originally general become complete poster
regulation means liberty rights state
better year last fan hitter
armenians troops armenia town brother
ready major although particularly mostly
clean race ask maintain member
help find need use try
prevent guarantee state powerful reduce
god faith christian heaven pray
letter review organize together frequently
bike article damn helmet yeah
write article get anyone paint
cap gary fan mike patrick
thanks advance please hus appreciate
ide isa hd apple scsi
arabs arab waco islamic israeli
kid laugh funny poor compound
california flight nasa institute activity
previously primary facility development additional
reach spend area prep

In [83]:

cost=[]
minibatches = create_minibatch(docs_te.astype('float32')[:7392])
total_batch = int(n_samples_tr / batch_size)
# Loop over all batches
for i in range(total_batch):
    batch_xs = minibatches.next()
    c=vae.test(batch_xs)
    n_d = np.sum(batch_xs)
    print n_d,c/n_d
    cost.append(c/n_d)
    
print (np.exp(np.mean(np.array(cost))))

16199.0 0.116587
17675.0 0.109971
18777.0 1.26609
17442.0 0.110946
15009.0 0.123144
15909.0 0.118173
18313.0 0.107068
12130.0 0.14373
16855.0 0.113791
15659.0 0.11908
15464.0 0.119873
17277.0 0.111419
14149.0 0.128386
16921.0 0.113744
18644.0 0.106128
14920.0 0.123464
17499.0 0.110225
13722.0 0.130724
18477.0 0.106227
17703.0 0.110081
17120.0 0.112562
17441.0 1.54291
14657.0 0.124756
18285.0 0.106832
19759.0 1.36587
18497.0 0.106362
15005.0 0.12258
15945.0 0.117134
14745.0 0.124032
15818.0 0.11807
18630.0 0.105868
15471.0 0.120294
16040.0 0.117025
17460.0 0.115694
14946.0 0.123214
19375.0 0.103608
14271.0 0.127357
15567.0 0.119424
18622.0 0.105208
19180.0 0.105486
17573.0 0.109752
17800.0 1.51213
17002.0 0.112617
13314.0 0.134064
15109.0 0.122592
15292.0 0.121212
18751.0 0.106105
15352.0 0.120089
13684.0 0.131339
17678.0 0.109578
15344.0 0.120314
18686.0 0.105825
20209.0 0.0999587
15927.0 0.118681
14833.0 0.123994
13999.0 0.128813
1.23346


In [52]:
with open('nips.txt','w') as o:

    for doc in docs_tr:
        doc_o=[]
        j=-1
        for wc in doc:
            j+=1
            for i in range(int(wc)):
                doc_o.append(vocab[j].strip())
        o.write(" ".join(doc_o)+'\n')

    

In [26]:
pickle.dump(cost,open('cost','w'))


In [60]:
print (np.exp(np.mean(np.array(cost))))

1.12524


In [54]:
print cost[51:]

[0.13918528, 0.43915969, 185.49721, 0.18895037, 0.17840242]
